In [ ]:
import html2text
import requests
import json
import operator
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

In [ ]:
ELASTIC_URL = "http://localhost:9200/"
json_header = {"Content-Type": "application/json"}
MAX_URL_LENGTH = 10
#nltk.download('punkt')
#nltk.download('stopwords')

In [ ]:
response = requests.get("https://towardsdatascience.com/data-augmentation-experimentation-3e274504f04b")

In [ ]:
print(response.text)

In [ ]:
site_content = html2text.html2text(response.text)

In [ ]:
print(site_content)

In [ ]:
special_chars = {u'ä': 'ae', 
                 u'ü': 'ue', 
                 u'ß': 'ss', 
                 u'ö': 'oe'}

for special_char in special_chars:
    site_content = site_content.replace(unicode(special_char), special_chars[special_char])
    

In [ ]:
print(site_content)

In [ ]:
nltk_tokens = word_tokenize(site_content)

In [ ]:
cleared_tokens = []

custom_stopwords = set({'http', 'https', 'and','jpg', 'mehr', 'fuer', 'gif', 'der', 'die', 'das', 'wir', 'ihr', 'sie', 'es', 'und', 'in', 'den'})
merged_stopwords = custom_stopwords.union(stopwords.words('german'))
merged_stopwords = merged_stopwords.union(stopwords.words('english'))

for nltk_token in nltk_tokens:
    if nltk_token.isalnum() and nltk_token.lower() not in merged_stopwords and len(nltk_token) > 2:
        cleared_tokens.append(nltk_token.lower())

In [ ]:
print(cleared_tokens)

In [ ]:
hits = {}

In [ ]:
for token in cleared_tokens:
    if token in hits:
        hits[token] = hits[token] +1
    else:
        hits[token] = 1

In [ ]:
for hit in hits:
    hits[hit] = hits[hit] / (len(hit)/2)

In [ ]:
sorted_hits = sorted(hits.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
sorted_hits

In [ ]:
found_url = False
i = 0
j = 1
repeat = 0
while not found_url:
    repeat = repeat +1
    
    if repeat > 7:
        found_url = True
        
    url = sorted_hits[i][0] + "-" + sorted_hits[j][0]
    url_reverse =  sorted_hits[j][0] + "-" + sorted_hits[i][0]
    print(url)
    print(url_reverse)
    
    if j-i > 3:
        i = i + 1
    else: 
        j = j + 1

In [ ]:
url

# Elastic

In [ ]:
def tokenize(text , analyzer):
    body = { 
        "analyzer": analyzer,
        "text": text
    }
    response = requests.get(ELASTIC_URL + '_analyze', data=json.dumps(body),headers = json_header).json()
    tokens = []
    for token in response['tokens']:
        tokens.append(token['token'])
    return tokens 

In [ ]:
tokens = tokenize(site_content, "german")